A. Research Question

- The posed research question, "To what extent does pre-release and early-release information such as platform, genre, developer, and initial reviews affect overall user ratings of a video game?", is a justifiable catalyst for research within the context of the video game industry, which has significant cultural and economic impact. Understanding these factors can assist in optimizing resources and better planning for market strategies.Gaining predictive capability for a game’s reception based on early indicators can be highly beneficial to stakeholders in the industry. The hypothesis that pre-release and early-release information, such as platform, genre, developer, and initial reviews, significantly affect the overall user ratings of a video game supports the potential to achieve this predictive capability.

B. Data Collection
- The publicly available dataset sourced from Kaggle contains Metacritic review data, including game titles, genres, developers, release dates, platforms, ratings, and reviews. One advantage of the dataset is that it is fairly comprehensive and structured in a manner that does not require extensive preprocessing, which facilitates easier analysis and model creation. However, a disadvantage of the dataset is its lack of the most recent video game releases, which could affect the relevance and accuracy of the predictions. A challenge that was overcome involved ensuring the data's completeness and quality by performing thorough preprocessing and data cleaning to handle inconsistencies and missing values.

C. Data Extraction and Preparation (Summary)
- - The dataset was loaded using Dask for efficient parallel processing.
- - Duplicate records were dropped and missing values were either imputed or dropped
- - Where necessary, data types were converted to a usable number format, and additional features such as release year, quarter, month, week, day of the week, and day of the year were extracted from the release date.
- - Review texts were preprocessed (stopwords removed, casing lowered, punctuation removed), and they were tokenized in padded sequences to ensure consistent shape for the model.
- - Categorical variables such as genre, platforms, game developer, and product rating were binarized using MultiLabelBinarizer, ensuring consistent shape for the model.
- - Numerical variables were scaled using StandardScaler.
- - Tools: Python, Dask, Pandas, Numpy, TensorFlow, Keras, sklearn
- - Techniques: Data cleaning, text preprocessing, feature extraction, tokenization, scaling
- These tools and techniques justifiably offer efficiency and effectiveness in handling complex computations and large datasets. While Dask allows parallel processing for faster data handling, TensorFlow and Keras are suitable for building neural network models. One advantage of using these tools and techniques is the capability to quickly and efficiently process large quantities of data. A disadvantage is the complexity involved in setting up and managing these tools, particularly parallel processing (Dask), which requires a solid understanding of their functionality and potential pitfalls.

D. Analysis (Summary)
- For analysis, a neural network model using TensorFlow and Keras was created and incrementally trained. To comprehensively capture the data present in the metacritic reviews, the model architecture included separate branches for numerical and text inputs that were later concatenated and fed into dense layers for prediction. To evaluate the model’s performance, Mean Squared Error (MSE), Mean Absolute Error (MAE), and R-Squared (R2) were used. A neural network was justifiably selected for analysis due to its ability to effectively handle both numerical and text data to capture complex patterns. An advantage of using a neural network is its flexibility, while a disadvantage is the high time required and computational cost for training the model.

C. Data Extraction and Preparation / D. Analysis (Detailed)
- Regex, NLTK, LangDetect, and LangID were used for text preprocessing, stopword removal, and language detection.
- Dask, Psutil were used for parallel processing of the dataset and memory usage optimization.
- Pandas, Collections, and Numpy were used for data manipulation and numerical operations.
- Tensflow and Keras were used for building and training the neural network model.
- sklearn was used for processing and model evaluation
- Pickle was used for saving and loading variables

In [3]:
import re as regex
import nltk as nltk
import langdetect as ldetect
import langid as lId
import psutil as psutil
import dask.dataframe as daskdf
import dask.distributed as daskdist
import dask.cache as DaskCache
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras.preprocessing as tfkpp
import tensorflow.keras.models as tfkm
import tensorflow.keras.layers as tfkl
import tensorflow.keras.callbacks as tfkc
import gc as gc
import io as io
import sklearn.preprocessing as skpp
import sklearn.model_selection as skms
import sklearn.metrics as skm
import collections as colls
import pickle as pkl

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\derek\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- The GetUniqueLabels() function extracts the unique labels within the entire dataset for the categorical variables, Genre, Platforms, Game Developer, and Product Rating. While all unique labels are used for Genre, Platforms, and Product Rating, only Game Developers that are present within more than 100 records are kept as unique labels; this was done because the infrequent labels greatly increased the processed dataset size while not being very significant for training.

In [4]:
def GetUniqueLabels():
    mcData = daskdf.read_csv(mcPath, low_memory = False, dtype = mcDaskDT)
    mcData = mcData[["Genre", "Platforms", "Game Developer", "Product Rating"]].dropna()
    mcData["Genre"] = mcData["Genre"].str[9:]
    devCounter = colls.Counter(list(mcData["Game Developer"].str.split(",").explode().compute()))
    devFilter = {dev for dev, c in devCounter.items() if c > 100}
    return {
        "Genre": list(mcData["Genre"].str.split(",").explode().unique().compute()),
        "Platforms": list(mcData["Platforms"].str.split(",").explode().unique().compute()),
        "Game Developer": list(devFilter),
        "Product Rating": list(mcData["Product Rating"].unique().compute())
    }

- The GetTokenVars() function calculates the length the sequences will be and extracts all unique english words within the entire dataset.

In [5]:
def GetTokenVars():
    mcData = daskdf.read_csv(mcPath, low_memory = False, dtype = mcDaskDT)
    mcData = mcData["Review Text"].dropna()
    def GetReviewLen(rev):
        if pd.isna(rev) or not rev.strip():
            return 0, []
        rWords = regex.findall(r"\b\w+\b", rev.lower())
        rWords = [word for word in rWords if word not in langPro["nltkSW"]]
        if not rWords:
            return 0, []
        try:
            jStr = (" ".join(rWords))
            langCheck1 = langPro["ldect"](jStr)
            langCheck2 = langPro["lId"](jStr)[0]
            if langCheck1 != "en" and langCheck2 != "en":
                return 0, []
        except:
            return 0, []
        return len(rWords), rWords
    lensAndWords = mcData.apply(GetReviewLen, meta = pd.Series(dtype = object)).compute()
    maxSeqLen = max(lensAndWords.map(lambda x: x[0]))
    allWords = [word for words in lensAndWords.map(lambda x: x[1]) for word in words]
    allWords = list(set(allWords))
    return maxSeqLen, allWords


- The ReviewTextPrep() function preprocesses review text by removing punctuation, lowering the casing, removing stopwords, and tokenzing, or it will mark review texts for removal if they are detected as non-english or are empty after preprocessing.

In [6]:
def ReviewTextPrep(rTxt, langPro, mcTokenizer, maxSeqLen):
    try:
        langCheck1 = langPro["ldect"](rTxt)
        langCheck2 = langPro["lId"](rTxt)[0]
        if langCheck1 == "en" or langCheck2 == "en":
            rTxt = rTxt.lower()
            rTxt = regex.sub(r"[^\w\s]", "", rTxt)
            rTxt = regex.sub(r"\d+", "", rTxt)
            rTxt = rTxt.split()
            rTxt = [word for word in rTxt if word not in langPro["nltkSW"]]
            if not rTxt:
                return "~NE~"
            rTxt = mcTokenizer.texts_to_sequences([rTxt])
            rTxt = tfkpp.sequence.pad_sequences(rTxt, maxlen = maxSeqLen, padding = "post")
            rTxt = np.clip(rTxt, 0, mcTokenizer.num_words - 1)
            return rTxt[0]
        else:
            return "~NE~"
    except:
        return "~NE~"

- The PreprocessData() function proprocesses all non-text data for the current dataset chunk by performing the following: 
- - Drops duplicate records
- - Drops missing value records for most columns ("Review Text", "Overall User Rating", "Overall Metascore", "Game Developer", "Rating Given By The Reviewer")
- - Imputes "Product Rating" missing values with its mode
- - Drops "Overall User Rating" values containing "tbd" rather than a numeric value, then coverts column to numeric format
- - Feature engineers "Game Release Date" into "Release Year", "Release Quarter", "Release Month", "Release Week", "Release DayOfWeek", and "Release DayOfYear"
- - Binarizes "Genre", "Platforms", "Game Developer", and "Product Rating" with Multilabel Binarizer to ensure consistent shape during one-hot encoding

In [7]:
def PreprocessData(mcDataChunk, uniqueLabels):
    allVars = ["Game Release Date", "Game Developer", "Genre", "Platforms", "Product Rating", "Overall Metascore", "Overall User Rating", "Rating Given By The Reviewer", "Review Text"]
    mcDataChunk = mcDataChunk[allVars]
    mcDataChunk = mcDataChunk.drop_duplicates()
    mcDataChunk = mcDataChunk.dropna(subset = ["Review Text", "Overall User Rating", "Overall Metascore", "Game Developer", "Rating Given By The Reviewer"])
    mcDataChunk["Product Rating"] = mcDataChunk["Product Rating"].fillna(mcDataChunk["Product Rating"].mode().compute()[0])
    mcDataChunk = mcDataChunk[mcDataChunk["Overall User Rating"] != "tbd"]
    mcDataChunk["Overall User Rating"] = mcDataChunk["Overall User Rating"].astype(np.float32)
    mcDataChunk["Game Release Date"] = daskdf.to_datetime(mcDataChunk["Game Release Date"], format = "%d-%b-%y", errors = "coerce")
    mcDataChunk = mcDataChunk.dropna(subset = ["Game Release Date"])
    mcDataChunk["Release Year"] = mcDataChunk["Game Release Date"].dt.year
    mcDataChunk["Release Quarter"] = mcDataChunk["Game Release Date"].dt.quarter
    mcDataChunk["Release Month"] = mcDataChunk["Game Release Date"].dt.month
    mcDataChunk["Release Week"] = np.floor(mcDataChunk["Game Release Date"].dt.day / 10).astype(int)
    mcDataChunk["Release DayOfWeek"] = mcDataChunk["Game Release Date"].dt.day_of_week
    mcDataChunk["Release DayOfYear"] = mcDataChunk["Game Release Date"].dt.day_of_year
    mcDataChunk = mcDataChunk.drop(["Game Release Date"], axis = 1)
    mcDataChunk["Genre"] = mcDataChunk["Genre"].str[9:]
    mcDataChunk["Genre"] = mcDataChunk["Genre"].str.split(",")
    mcDataChunk["Platforms"] = mcDataChunk["Platforms"].str.split(",")
    mcDataChunk["Game Developer"] = mcDataChunk["Game Developer"].str.split(",")
    mcDataChunk["Product Rating"] = mcDataChunk["Product Rating"].str.split(",")
    mlbGenre = skpp.MultiLabelBinarizer(classes = uniqueLabels["Genre"])
    mlbPlatforms = skpp.MultiLabelBinarizer(classes = uniqueLabels["Platforms"])
    mlbDeveloper = skpp.MultiLabelBinarizer(classes = uniqueLabels["Game Developer"])
    mlbRating = skpp.MultiLabelBinarizer(classes = uniqueLabels["Product Rating"])
    mlbGenreData = mlbGenre.fit_transform(mcDataChunk["Genre"].compute())
    mlbPlatformsData = mlbPlatforms.fit_transform(mcDataChunk["Platforms"].compute())
    mlbDeveloperData = mlbDeveloper.fit_transform(mcDataChunk["Game Developer"].compute())
    mlbRatingData = mlbRating.fit_transform(mcDataChunk["Product Rating"].compute())
    genrePDF = pd.DataFrame(mlbGenreData, columns = mlbGenre.classes_, index = mcDataChunk.index.compute())
    platformsPDF = pd.DataFrame(mlbPlatformsData, columns = mlbPlatforms.classes_, index = mcDataChunk.index.compute())
    developerPDF = pd.DataFrame(mlbDeveloperData, columns = mlbDeveloper.classes_, index = mcDataChunk.index.compute())
    ratingPDF = pd.DataFrame(mlbRatingData, columns = mlbRating.classes_, index = mcDataChunk.index.compute())
    mcDataChunk = mcDataChunk.drop(["Genre", "Platforms", "Game Developer", "Product Rating"], axis = 1)
    mcDataChunk = mcDataChunk.join(daskdf.from_pandas(genrePDF, npartitions = 1))
    mcDataChunk = mcDataChunk.join(daskdf.from_pandas(platformsPDF, npartitions = 1))
    mcDataChunk = mcDataChunk.join(daskdf.from_pandas(developerPDF, npartitions = 1))
    mcDataChunk = mcDataChunk.join(daskdf.from_pandas(ratingPDF, npartitions = 1))
    daskdist.wait(mcDataChunk)
    del mlbGenre, mlbPlatforms, mlbDeveloper, mlbRating, mlbGenreData, mlbPlatformsData, mlbDeveloperData, mlbRatingData, genrePDF, platformsPDF, developerPDF, ratingPDF
    gc.collect()
    return mcDataChunk

- The FinalPreprocessAndTrain() function completes the current dataset chunk preprocessing and starts model training by performing the following:
- - Preprocessess "Review Text" into sequences using the ReviewTextPrep() function, then drops all records marked for removal by the function
- - Scales numerical columns
- - Splits dataset into training and testing sets
- - Creates TensorFlow datasets for training and testing sets
- - Fits model with TensorFlows datasets

In [8]:
def FinalPreprocessAndTrain(mcDataChunk, langPro, mcMod, mcTokenizer, maxSeqLen, mcScaler):
    mcDataChunk["Review Text"] = mcDataChunk["Review Text"].map_partitions(lambda mcDF: mcDF.apply(ReviewTextPrep, langPro = langPro, mcTokenizer = mcTokenizer, maxSeqLen = maxSeqLen), meta = pd.Series(dtype = "object", name = "Review Text"))
    mcDataChunk = mcDataChunk[mcDataChunk["Review Text"].apply(lambda x: "~NE~" not in x, meta = ("Review Text", "bool"))]
    mcDataChunk = mcDataChunk.compute()
    mcDataChunk = mcDataChunk.reset_index(drop = True)
    mcSeqs = np.stack(mcDataChunk["Review Text"].values)
    mcDataChunk = mcDataChunk.drop(["Review Text"], axis = 1)
    mcDataChunk = mcDataChunk.join(pd.DataFrame(mcSeqs, index = mcDataChunk.index))
    mcY = mcDataChunk["Overall Metascore"].copy().astype(np.float32)
    mcX = mcDataChunk.copy().drop(["Overall Metascore"], axis = 1).astype(np.float32)
    del mcSeqs, mcDataChunk
    gc.collect()
    mcXTrain, mcXTest, mcYTrain, mcYTest = skms.train_test_split(mcX, mcY, test_size = 0.2, random_state = 17)
    numVars = ["Overall User Rating", "Release Year", "Release Quarter", "Release Month", "Release Week", "Release DayOfWeek", "Release DayOfYear"]
    mcXTrain[numVars] = mcScaler.transform(mcXTrain[numVars])
    mcXTest[numVars] = mcScaler.transform(mcXTest[numVars])
    mcXTrain = mcXTrain.to_numpy()
    mcXTest = mcXTest.to_numpy()
    mcTFDataTrain = tf.data.Dataset.from_tensor_slices(((mcXTrain[:, -maxSeqLen:], mcXTrain[:, :-maxSeqLen]), mcYTrain.values)).batch(32)
    mcTFDataTest = tf.data.Dataset.from_tensor_slices(((mcXTest[:, -maxSeqLen:], mcXTest[:, :-maxSeqLen]), mcYTest.values)).batch(32)
    mcMod.fit(mcTFDataTrain, validation_data = mcTFDataTest, epochs = 20, callbacks = [tfkc.EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)])
    return mcMod

- The SaveVar(), LoadVar(), SaveModVars(), and LoadModVars() functions are utility functions for saving and loading variables are .pkl files.

In [9]:
def SaveVar(var, fName):
    with open(f"intermediate/{fName}.pkl", "wb") as wb:
        pkl.dump(var, wb)

def LoadVar(fName):
    with open(f"intermediate/{fName}.pkl", "rb") as rb:
        return pkl.load(rb)
    
def SaveModVars():
    SaveVar(mcScaler, "../mcScaler")
    SaveVar(mcTokenizer, "../mcTokenizer")
    SaveVar(maxSeqLen, "../maxSeqLen")
    SaveVar(uniqueLabels, "../uniqueLabels")

def LoadModVars():
    with open("mcScaler.pkl", "rb") as rb:
        mcScaler = pkl.load(rb)
    with open("mcTokenizer.pkl", "rb") as rb:
        mcTokenizer = pkl.load(rb)
    with open("maxSeqLen.pkl", "rb") as rb:
        maxSeqLen = pkl.load(rb)
    with open("uniqueLabels.pkl", "rb") as rb:
        uniqueLabels = pkl.load(rb)
    return mcScaler, mcTokenizer, maxSeqLen, uniqueLabels

- The EvaluateMCModel() function generates MSE, MAE, and R2 values based on a sample of the dataset.

In [10]:
def EvaluateMCModel(mcMod, mcScaler, mcTokenizer, maxSeqLen, uniqueLabels):
    mcDataChunk = daskdf.read_csv(mcPath, low_memory = False, dtype = mcDaskDT).sample(frac = 0.01)
    mcDataChunk = PreprocessData(mcDataChunk, uniqueLabels)
    mcDataChunk["Review Text"] = mcDataChunk["Review Text"].map_partitions(lambda mcDF: mcDF.apply(ReviewTextPrep, langPro = langPro, mcTokenizer = mcTokenizer, maxSeqLen = maxSeqLen), meta = pd.Series(dtype = "object", name = "Review Text"))
    mcDataChunk = mcDataChunk[mcDataChunk["Review Text"].apply(lambda x: "~NE~" not in x, meta = ("Review Text", "bool"))]
    mcDataChunk = mcDataChunk.compute()
    mcDataChunk = mcDataChunk.reset_index(drop = True)
    mcSeqs = np.stack(mcDataChunk["Review Text"].values)
    mcDataChunk = mcDataChunk.drop(["Review Text"], axis = 1)
    mcDataChunk = mcDataChunk.join(pd.DataFrame(mcSeqs, index = mcDataChunk.index))
    numVars = ["Overall User Rating", "Release Year", "Release Quarter", "Release Month", "Release Week", "Release DayOfWeek", "Release DayOfYear"]
    mcDataChunk[numVars] = mcScaler.transform(mcDataChunk[numVars])
    mcY = mcDataChunk["Overall Metascore"].copy().astype(np.float32).to_numpy()
    mcX = mcDataChunk.copy().drop(["Overall Metascore"], axis = 1).astype(np.float32).to_numpy()
    del mcSeqs, mcDataChunk
    gc.collect()
    mcTestText = mcX[:, -maxSeqLen:]
    mcTestNum = mcX[:, :-maxSeqLen]
    mcTestPreds = mcMod.predict([mcTestText, mcTestNum])
    mcMSE = skm.mean_squared_error(mcY, mcTestPreds)
    mcMAE = skm.mean_absolute_error(mcY, mcTestPreds)
    mcR2 = skm.r2_score(mcY, mcTestPreds)
    return mcMSE, mcMAE, mcR2

- The PredictMetascore() function predicts an "Overall Metascore" value using the model.

In [11]:
def PredictMetascore(mcMod, mcTokenizer, mcScaler, maxSeqLen, inData):
    inData = pd.DataFrame([inData])
    inData = PreprocessData(daskdf.from_pandas(inData, npartitions = 1), uniqueLabels)
    inData["Review Text"] = inData["Review Text"].map_partitions(lambda mcDF: mcDF.apply(ReviewTextPrep, langPro = langPro, mcTokenizer = mcTokenizer, maxSeqLen = maxSeqLen), meta = pd.Series(dtype = "object", name = "Review Text"))
    inData = inData[inData["Review Text"].apply(lambda x: "~NE~" not in x, meta = ("Review Text", "bool"))]
    inData = inData.compute()
    inData = inData.reset_index(drop = True)
    mcSeqs = np.stack(inData["Review Text"].values)
    inData = inData.drop(["Review Text"], axis = 1)
    inData = inData.join(pd.DataFrame(mcSeqs, index = inData.index))
    numVars = ["Overall User Rating", "Release Year", "Release Quarter", "Release Month", "Release Week", "Release DayOfWeek", "Release DayOfYear"]
    inData[numVars] = mcScaler.transform(inData[numVars])
    inX = inData.copy().drop(["Overall Metascore"], axis = 1).astype(np.float32).to_numpy()
    del mcSeqs, inData
    gc.collect()
    inText = inX[:, -maxSeqLen:]
    inNum = inX[:, :-maxSeqLen]
    mcPred = mcMod.predict([inText, inNum])
    return mcPred

- The langPro dictionary contains functions for detecting languages, as well as the set of English stopwords.
- The mcDaskDT dictionary is used for reference for the dataset dtypes while creating dask dataframes.

In [12]:
ldetect.DetectorFactory.seed = 17
langPro = {
    "ldect": ldetect.detect,
    "lId": lId.langid.classify,
    "nltkSW": set(nltk.corpus.stopwords.words("english"))
}
mcDaskDT = {
    "Review Date": "object",
    "Overall User Rating": "object"
}

- The number of records in the dataset is calculated and the headers are extracted

In [13]:
mcPath = "metacritic_pc_games.csv"
rootChunkSize = 10000
with io.open(mcPath, "r", encoding = "utf-8", buffering = 1024**3) as mcFile:
    mcHeaders = mcFile.readline().strip().split(",")
    mcEntryCount = sum(1 for line in mcFile) - 1

- Dask is initialized with a configuration optimized for initial dataset processing

In [14]:
chunkCount = psutil.cpu_count()
memUtil = 0.75
daskCache = DaskCache.Cache(2e9)
daskCache.register()
daskClient = daskdist.Client(n_workers = 2, threads_per_worker = 4, memory_limit = f"{int(psutil.virtual_memory().total / (1024 ** 2) * memUtil / 2)}MB")

- All unique English words are extracted from the entire dataset and the maximum sequence length is calculated.

In [15]:
maxSeqLen, allWords = GetTokenVars()
SaveVar(maxSeqLen, "maxSeqLen")
SaveVar(allWords, "allWords")

- Dask is reinitialized with a configuration optimized for the remaining operations

In [16]:
daskClient.shutdown()
daskClient = daskdist.Client(n_workers = chunkCount, threads_per_worker = 2, memory_limit = f"{int(psutil.virtual_memory().total / (1024 ** 2) * memUtil / chunkCount)}MB")

- Unique labels are extracted

In [17]:
uniqueLabels = GetUniqueLabels()
SaveVar(uniqueLabels, "uniqueLabels")

- Scaler is initialized with the numerical data of a dataset sample

In [18]:
initChunk = daskdf.read_csv(mcPath, low_memory = False, dtype = mcDaskDT)
initChunk = PreprocessData(initChunk.sample(frac = 0.01), uniqueLabels)
numVars = ["Overall User Rating", "Release Year", "Release Quarter", "Release Month", "Release Week", "Release DayOfWeek", "Release DayOfYear"]
mcScaler = skpp.StandardScaler().fit(initChunk[numVars].compute())
SaveVar(mcScaler, "mcScaler")

y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) ['Text'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) [' 1C-Entertainment ', ' 1C:Ino-Co ', ' 343 Industries ', ' Arzest', ' Atlus ', ' Balan Company ', ' Beamdog', ' Bitworks ', ' Black Sea Games ', ' BlackMuffin Studio', ' Blitz Games ', ' Boss Key Productions ', ' Clover Bite ', ' Co. LTD ', ' Cognosphere ', ' Creative Business Unit III ', " DON'T NOD ", ' Davit Andreasyan ', ' Digital Sun ', ' Dlala Studios ', ' Double Damage Games ', ' Drool ', ' EA Games ', ' EA Vancouver ', ' Easy Trigger Games ', ' Eighting ', ' Endnight Studios ', ' Flashback Games ', ' Florian Himsl ', ' Foreign Gnomes ', ' Goldhawk Interactive ', ' Gone North Games ', ' Grove Street Games ', ' Gun Interactive ', ' Hopoo Games ', ' Hotta Stud

- Tokenizer is initialized with all unique English words in dataset

In [19]:
mcTokenizer = tfkpp.text.Tokenizer(num_words = len(allWords) + 1)
mcTokenizer.fit_on_texts(allWords)
SaveVar(mcTokenizer, "mcTokenizer")
vocabSize = len(mcTokenizer.word_index) + 1

- The model is built with an architecture consisting of two input branches (one text branch and one numeric branch) that are concatenated and fed into dense layers.

In [20]:
numShape = len(numVars) + initChunk.drop(["Review Text", "Overall Metascore"] + numVars, axis = 1).shape[1]
inShape = maxSeqLen + numShape
#text branch
textInLayer = tfkl.Input(shape = (maxSeqLen,), name = "textInLayer")
embedLayer = tfkl.Embedding(input_dim = vocabSize, output_dim = 128)(textInLayer)
lstmLayer = tfkl.LSTM(64)(embedLayer)
dropoutLayer = tfkl.Dropout(0.5)(lstmLayer)
#numeric branch
numInLayer = tfkl.Input(shape = (numShape,), name = "numInLayer")
dense1Layer = tfkl.Dense(64, activation = "relu")(numInLayer)
dropout1Layer = tfkl.Dropout(0.5)(dense1Layer)
#concatenated output
outLayerConcat = tfkl.concatenate([dropoutLayer, dropout1Layer])
#dense layers
dense2Layer = tfkl.Dense(32, activation = "relu")(outLayerConcat)
outLayer = tfkl.Dense(1, activation = "linear")(dense2Layer)
#compiling model
mcMod = tfkm.Model(inputs=[textInLayer, numInLayer], outputs=outLayer)
mcMod.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ["mae"])
mcMod.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
textInLayer (InputLayer)        [(None, 2171)]       0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 2171, 128)    32820096    textInLayer[0][0]                
__________________________________________________________________________________________________
numInLayer (InputLayer)         [(None, 735)]        0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 64)           49408       embedding[0][0]                  
______________________________________________________________________________________________

- The model is incrementally trained in chunks of a maximum of 10,000 records at a time. The model is trained on the entire dataset 3 times, with early stopping implemented to avoid overfitting.

In [21]:
trainingCycles = 3
for trainingCycle in range(trainingCycles):
    chunkLoopCount = int(np.ceil(mcEntryCount / rootChunkSize))
    for rootChunkLoopCounter in range(chunkLoopCount):
        gc.collect()
        print(f"Chunk {rootChunkLoopCounter + 1}/{chunkLoopCount} // {np.round((rootChunkLoopCounter / chunkLoopCount) * 100, 2)}% Complete // Cycle {trainingCycle + 1}/{trainingCycles}")
        if rootChunkLoopCounter != 0:
            mcDataChunk = PreprocessData(daskdf.from_pandas(pd.read_csv(mcPath, names = mcHeaders, chunksize = rootChunkSize, skiprows = rootChunkSize * rootChunkLoopCounter, low_memory = False).get_chunk(rootChunkSize), npartitions = chunkCount), uniqueLabels)
        else:
            mcDataChunk = PreprocessData(daskdf.from_pandas(pd.read_csv(mcPath, chunksize = rootChunkSize, low_memory = False).get_chunk(rootChunkSize), npartitions = chunkCount), uniqueLabels)
        mcMod = FinalPreprocessAndTrain(mcDataChunk, langPro, mcMod, mcTokenizer, maxSeqLen, mcScaler)
        mcMod.save("intermediate/MetacriticReviewModel_chunkCheckpoint.keras")
    mcMod.save("intermediate/MetacriticReviewModel_cycleCheckpoint.keras")
mcMod.save("MetacriticReviewModel.keras")
SaveModVars()

Chunk 1/52 // 0.0% Complete // Cycle 1/3


y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) [' Arzest', ' Atlantis Interactive Entertainment ', ' Avalanche Software ', ' Balan Company ', ' Bird Bath Games ', ' Cape Guy ', ' Dlala Studios ', ' Eurocom ', ' Forgotten Key ', ' Gone North Games ', ' Innersloth ', ' Massive Entertainment', ' Viva Media', ' bitComposer ', ' iNK Stories ', '2K Games ', '3000AD', '3D People ', 'A44 ', 'AGEOD ', 'AWE Games ', 'Accidental Queens ', 'Acclaim Studios Cheltenham ', 'Acme Gamestudio ', 'Aeternum Game Studios ', 'Alan Zucconi ', 'Alientrap ', 'Analgesic Productions ', 'Anarteam ', 'Arcen Games', 'Argonaut Games ', 'Armature Studio ', 'Artematica ', 'Artisan Studios ', 'Arxel Tribe ', 'Atlantis Interactive Entertainment\t', 'AtomTeam ', 'Auran ', 'Auroch Digital ', 'Aurogon Shanghai ', 'AurumDust ', 'Balancing Monkey Games ', 'Bedtime Digital Games ', 'Benjamin Rivers ', 'BestGameEver.com ', 'Big Ant Studios ', 'Big Boat Interacti

Epoch 1/20
239/239 [==============================] - 28s 100ms/step - loss: 800.2854 - mae: 19.6956 - val_loss: 59.9708 - val_mae: 6.0429
Epoch 2/20
239/239 [==============================] - 24s 101ms/step - loss: 140.1887 - mae: 9.4075 - val_loss: 52.3805 - val_mae: 5.6460
Epoch 3/20
239/239 [==============================] - 24s 102ms/step - loss: 115.3553 - mae: 8.5566 - val_loss: 48.0988 - val_mae: 5.4418
Epoch 4/20
239/239 [==============================] - 24s 100ms/step - loss: 106.8944 - mae: 8.1932 - val_loss: 43.1095 - val_mae: 5.0839
Epoch 5/20
239/239 [==============================] - 24s 102ms/step - loss: 96.4165 - mae: 7.8114 - val_loss: 45.6866 - val_mae: 5.4084
Epoch 6/20
239/239 [==============================] - 25s 103ms/step - loss: 95.6736 - mae: 7.7754 - val_loss: 38.7117 - val_mae: 4.8660
Epoch 7/20
239/239 [==============================] - 24s 102ms/step - loss: 90.6066 - mae: 7.5681 - val_loss: 34.7035 - val_mae: 4.4379
Epoch 8/20
239/239 [================

y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) ['Application', 'Cricket', 'Trainer'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) [' 10tons ', ' Cranberry Production ', ' Deep Silver Fishlabs ', ' Digital Eclipse ', ' EA Games ', ' Elbow Rocket ', ' FAKT Software ', ' FireFly Studios ', ' Funatics ', ' Funatics Software', ' ICEBOX Studios ', ' LLC', ' LandFall ', ' Moonbyte Studios ', ' Navarre Corp ', ' Nival ', ' Out of the Blue Games S.L. ', ' Overflow Games ', ' Phosphor Games ', ' Polar Motion ', ' RA Images ', ' Reality Twist ', ' Relentless Studios ', ' S.L. ', ' Sega', ' Slitherine Group', ' Ste Curran ', '10tons Entertainment', '369 Interactive ', 'A Crowd of Monsters ', 'Aarne "MekaSkull" Hunziker ', 'Access Games ', 'Action Forms Ltd. ', 'Activision ', '

Epoch 1/20
238/238 [==============================] - 24s 100ms/step - loss: 94.3689 - mae: 7.7100 - val_loss: 39.3056 - val_mae: 4.9422
Epoch 2/20
238/238 [==============================] - 25s 103ms/step - loss: 84.3888 - mae: 7.3517 - val_loss: 36.7194 - val_mae: 4.7872
Epoch 3/20
238/238 [==============================] - 25s 104ms/step - loss: 81.8371 - mae: 7.2394 - val_loss: 38.8149 - val_mae: 5.0397
Epoch 4/20
238/238 [==============================] - 24s 102ms/step - loss: 75.5576 - mae: 6.8854 - val_loss: 31.9870 - val_mae: 4.3556
Epoch 5/20
238/238 [==============================] - 24s 101ms/step - loss: 76.7502 - mae: 6.9760 - val_loss: 34.3535 - val_mae: 4.6751
Epoch 6/20
238/238 [==============================] - 24s 101ms/step - loss: 74.0961 - mae: 6.8318 - val_loss: 30.0747 - val_mae: 4.2472
Epoch 7/20
238/238 [==============================] - 23s 98ms/step - loss: 69.2829 - mae: 6.6282 - val_loss: 31.1353 - val_mae: 4.3976
Epoch 8/20
238/238 [======================

...

Epoch 1/20
228/228 [==============================] - 17s 75ms/step - loss: 22.3174 - mae: 3.0354 - val_loss: 22.6371 - val_mae: 3.8165
Epoch 2/20
228/228 [==============================] - 17s 75ms/step - loss: 15.2918 - mae: 2.3999 - val_loss: 24.0463 - val_mae: 4.0801
Epoch 3/20
228/228 [==============================] - 17s 75ms/step - loss: 12.2950 - mae: 2.0961 - val_loss: 23.2835 - val_mae: 3.9970
Epoch 4/20
228/228 [==============================] - 17s 75ms/step - loss: 11.2509 - mae: 1.9793 - val_loss: 23.6823 - val_mae: 4.0594
Epoch 5/20
228/228 [==============================] - 17s 75ms/step - loss: 9.9033 - mae: 1.8182 - val_loss: 21.9559 - val_mae: 3.9548
Epoch 6/20
228/228 [==============================] - 17s 74ms/step - loss: 9.5943 - mae: 1.7453 - val_loss: 20.5055 - val_mae: 3.8025
Epoch 7/20
228/228 [==============================] - 17s 76ms/step - loss: 8.7207 - mae: 1.6818 - val_loss: 20.5078 - val_mae: 3.7649
Epoch 8/20
228/228 [==============================]

distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)


Chunk 52/52 // 98.08% Complete // Cycle 3/3


y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) [' Goldhawk Interactive ', ' KADOKAWA CORPORATION ', ' Lab42 ', ' Mandel ArtPlains ', ' Sumo Digital Academy ', ' comcept ', '1P2P ', 'Ackkstudios ', 'Baroque Decay ', 'Brave at Night ', 'Crocodile Entertainment ', 'Digital Spray Studios', 'EXOR Studios ', 'Gamedust ', 'Gamethunks', 'Impressions Games ', 'Jellyvision ', 'Kadokawa', 'Livesay Technologies ', 'MicroProse ', 'Moonbite Games ', 'NoClip ', 'PlayMagic Ltd ', 'Prideful Sloth ', 'Rimlight Studios ', 'Ruud Koorevaar ', 'Ryu ga Gotoku Studios', 'Simogo ', 'Spark Unlimited', 'Spike Chunsoft ', 'Sumo Digital', 'Unicorn Games Studio ', 'Villa Gorilla ', 'WXP ', 'frecle ApS '] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collect

Epoch 1/20
77/77 [==============================] - 6s 77ms/step - loss: 29.6564 - mae: 3.4187 - val_loss: 40.0114 - val_mae: 5.3970
Epoch 2/20
77/77 [==============================] - 6s 76ms/step - loss: 17.3319 - mae: 2.4767 - val_loss: 33.7028 - val_mae: 4.9802
Epoch 3/20
77/77 [==============================] - 6s 76ms/step - loss: 16.8575 - mae: 2.3113 - val_loss: 34.1649 - val_mae: 4.9958
Epoch 4/20
77/77 [==============================] - 6s 75ms/step - loss: 13.6008 - mae: 2.0189 - val_loss: 32.6427 - val_mae: 5.0028
Epoch 5/20
77/77 [==============================] - 6s 75ms/step - loss: 11.8931 - mae: 1.8864 - val_loss: 32.0052 - val_mae: 4.9696
Epoch 6/20
77/77 [==============================] - 6s 75ms/step - loss: 11.2721 - mae: 1.8535 - val_loss: 32.8839 - val_mae: 5.0433
Epoch 7/20
77/77 [==============================] - 6s 74ms/step - loss: 11.2571 - mae: 1.7990 - val_loss: 31.6614 - val_mae: 4.9345
Epoch 8/20
77/77 [==============================] - 6s 76ms/step - lo

E. Data Summary and Implications
- The data analysis revealed significant relationships between pre-release and early-release information and the overall user ratings. The model showed decent predictive performance, with reasonable values for MSE, MAE, and R2 that suggest that although the model has room for improvement, it can provide useful predictions. One limitation of the analysis is that it relies on historical data, some of which is somewhat dated now, so it may not fully capture evolving trends in the video game industry. It is recommended that the model be further refined, so that game developers and publishers can use the model to optimize pre-release and early-release strategies, such as improving initial reviews and targeting popular genres and platforms, to enhance overall user reception.
- Future studies could include social media sentiment analysis to capture more real-time and dynamic user opinions.
- Additionally, expanding the dataset to include more recent games and additional platforms could improve the model's relevance and accuracy.

In [22]:
mcMSE, mcMAE, mcR2 = EvaluateMCModel(mcMod, mcScaler, mcTokenizer, maxSeqLen, uniqueLabels)
print(f"Evaluation -- MSE: {mcMSE}, MAE: {mcMAE}, R2: {mcR2}")

y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) ['Text'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) [' 1C', ' 343 Industries ', ' Acclaim Studios Manchester ', ' Animation Arts ', ' Application Systems Heidelberg ', ' Atari SA ', ' B.B.Studio ', ' Behaviour Interactive ', ' Bigben Games ', ' Bigben Interactive', ' Bitworks ', ' Capcom Vancouver ', ' Cape Guy ', ' Cauldron Ltd. ', ' Cherrymochi ', ' Cognosphere ', ' Creative Business Unit III ', ' Crows Crows Crows ', ' Deep Silver Fishlabs ', ' Devoured Studios ', ' Digital Sun ', ' Dlala Studios ', ' Do My Best Games ', ' Dose', ' EA Singapore ', ' EA Vancouver ', ' Elbow Rocket ', ' Elemental Games', ' Endnight Studios ', ' Eurocom ', ' Flashback Games ', ' Florian Himsl ', ' Fullbright ', ' FunHouse ', ' Funati

Evaluation -- MSE: 119.35368347167969, MAE: 8.38284683227539, R2: 0.1009657202754819


In [24]:
inData = {
    "Game Release Date": "27-Feb-25",
    "Game Developer": "Nintendo",
    "Genre": "Genre(s):Miscellaneous,Puzzle,Action,Platformer",
    "Platforms": "PlayStation5,Switch,PC",
    "Product Rating": "T",
    "Overall Metascore": 80,
    "Overall User Rating": "8.5",
    "Rating Given By The Reviewer": "8",
    "Review Text": "The open-world gameplay is makes this platformer highly enjoyable."
}
mcPred = PredictMetascore(mcMod, mcTokenizer, mcScaler, maxSeqLen, inData)
print(f"Predicted Metascore: {mcPred[0][0]}")

y:\ML\Anaconda\envs\TF-GPU\lib\site-packages\sklearn\preprocessing\_label.py:876: UserWarning: unknown class(es) ['Nintendo'] will be ignored
  "unknown class(es) {0} will be ignored".format(sorted(unknown, key=str))
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


Predicted Metascore: 75.68212127685547
